In [1]:
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, AvgPool2d
from torch.optim import Adam, SGD
from network import AuctionNet, AucLoss, process
from tqdm import tqdm

In [2]:
# Sample. 4 players, 3 items
p1 = [8, 7, 6]
p2 = [12, 8, 4]
p3 = [9, 8, 7]
p4 = [11, 10, 7]

other_results, h_vcg, results, tensors = process(torch.tensor([p1, p2, p3, p4]))
print(tensors)

tensor([[[[12.,  8.,  4.],
          [ 9.,  8.,  7.],
          [11., 10.,  7.]],

         [[ 1.,  0.,  0.],
          [ 0.,  0.,  0.],
          [ 0.,  0.,  0.]],

         [[12.,  0.,  0.],
          [ 0.,  0.,  0.],
          [ 0.,  0.,  0.]],

         [[ 1.,  0.,  0.],
          [ 0.,  0.,  0.],
          [ 1.,  0.,  0.]],

         [[12.,  0.,  0.],
          [ 0.,  0.,  0.],
          [11.,  0.,  0.]],

         [[ 1.,  0.,  0.],
          [ 0.,  0.,  0.],
          [ 1.,  1.,  0.]],

         [[12.,  0.,  0.],
          [ 0.,  0.,  0.],
          [11., 10.,  0.]]],


        [[[ 8.,  7.,  6.],
          [ 9.,  8.,  7.],
          [11., 10.,  7.]],

         [[ 0.,  0.,  0.],
          [ 0.,  0.,  0.],
          [ 1.,  0.,  0.]],

         [[ 0.,  0.,  0.],
          [ 0.,  0.,  0.],
          [11.,  0.,  0.]],

         [[ 0.,  0.,  0.],
          [ 0.,  0.,  0.],
          [ 1.,  1.,  0.]],

         [[ 0.,  0.,  0.],
          [ 0.,  0.,  0.],
          [11., 10.,  0.]],

  

In [3]:
vs = torch.normal(10, 1, [4, 3])
vs, indicies = torch.sort(vs, descending=True)
other_results, h_vcg, results, tensors = process(vs)

In [5]:
model = AuctionNet(4, 3)
optimizer = Adam(model.parameters(), lr=1e-3)
loss_func = AucLoss()
print(model(tensors))

tensor([[0.1494],
        [0.0924],
        [0.0592],
        [0.1312]], grad_fn=<AddmmBackward0>)


In [6]:
# This is obviously going to overfit, but just a show of it learning the function for one particular auction
for _ in range(100):
    optimizer.zero_grad()
    h = model(tensors)
    h = h.reshape(-1)
    t = h - other_results
    loss = loss_func(t, results)
    loss.backward()
    optimizer.step()

In [7]:
def loss_trial( t, results, lam_b=100, lam_r=100):
    sum_t = torch.sum(t)
    print(f'sum_t: {sum_t}')
    budget_balanced = torch.square(torch.minimum(torch.sum(t), torch.zeros([1])))
    print(f'budget_balanced: {budget_balanced}')
    rationality = torch.sum(torch.square(torch.minimum(results - t, torch.zeros(t.shape))))
    print(f'rationality: {rationality}')
    loss = sum_t + lam_b*budget_balanced + lam_r*rationality
    return loss


In [8]:
loss_trial(t, results)

sum_t: 7.8319902420043945
budget_balanced: tensor([0.], grad_fn=<PowBackward0>)
rationality: 0.0


tensor([7.8320], grad_fn=<AddBackward0>)

In [9]:
model(tensors)

tensor([[28.2851],
        [27.7180],
        [27.2737],
        [28.2134]], grad_fn=<AddmmBackward0>)

In [10]:
h_vcg

tensor([34.5496, 32.7032, 32.8398, 34.5496])

In [13]:
num_train_auctions = 20000
num_test_auctions = 2000
n_players = 10
n_items = 15

all_other_results, all_h_vcg, all_results, all_tensors = [], [], [], []

for _ in tqdm(range(num_train_auctions)):
    mean = torch.normal(10, 1, [1])
    sd = max(torch.normal(2, .5, [1]), torch.tensor(0))
    a = torch.empty((n_players, n_items)).normal_(mean.item(), sd.item())
    other_results, h_vcg, results, tensors = process(a)
    all_other_results.append(other_results)
    all_h_vcg.append(h_vcg)
    all_results.append(results)
    all_tensors.append(tensors)
    
    
test_other_results, test_h_vcg, test_results, test_tensors = [], [], [], []
    
for _ in tqdm(range(num_test_auctions)):
    mean = torch.normal(10, 1, [1])
    sd = max(torch.normal(2, .5, [1]), 0)
    a = torch.empty((n_players, n_items)).normal_(mean.item(), sd.item())
    other_results, h_vcg, results, tensors = process(a)
    test_other_results.append(other_results)
    test_h_vcg.append(h_vcg)
    test_results.append(results)
    test_tensors.append(tensors)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:17<00:00, 116.69it/s]


In [ ]:
n_epochs = 100000

model = AuctionNet(n_players, n_items)
optimizer = Adam(model.parameters(), lr=1e-3)
loss_func = AucLoss()


for e in range(n_epochs):
    loss = 0
    for i, t in enumerate(all_tensors):
        optimizer.zero_grad()
        h = model(t)
        h = h.reshape(-1)
        t = h - all_other_results[i]
        loss += loss_func(t, all_results[i])
        if i % 500 == 0 and i != 0:
            loss.backward()
            optimizer.step()
            loss = 0
    
    budget_violation = 0
    reduction = 0
    for i, t in enumerate(test_tensors):
        res = model(t)
        vgc = test_h_vcg[i].clone().detach()
        res = res.reshape(-1)

        res -= test_other_results[i]
        vgc -= test_other_results[i]
        res = torch.sum(res)
        vgc = torch.sum(vgc)

        if res < 0:
            budget_violation += 1

        reduction += 1 - res/vgc
    print(f'epoch {e}: {reduction/len(test_tensors)}, {budget_violation}')
    

epoch 0: -0.2641617953777313, 19
epoch 1: -0.24675412476062775, 21
epoch 2: 0.10229580849409103, 86
epoch 3: 0.16082265973091125, 104
epoch 4: 0.15727682411670685, 103
epoch 5: 0.1562829613685608, 99
epoch 6: 0.15708263218402863, 97
epoch 7: 0.15902778506278992, 95
epoch 8: 0.16202381253242493, 95
epoch 9: 0.17332126200199127, 94
epoch 10: 0.2140221893787384, 80
epoch 11: 0.24909549951553345, 79
epoch 12: 0.2598276734352112, 72
epoch 13: 0.2754030227661133, 64
epoch 14: 0.30921247601509094, 54
epoch 15: 0.3155415952205658, 49
epoch 16: 0.3286495506763458, 50
epoch 17: 0.33214104175567627, 48
epoch 18: 0.3530828654766083, 45
epoch 19: 0.34856948256492615, 40
epoch 20: 0.3510524034500122, 40
epoch 21: 0.35257548093795776, 39
epoch 22: 0.3536156117916107, 39
epoch 23: 0.3555085062980652, 38
epoch 24: 0.3568221628665924, 39
epoch 25: 0.3582336902618408, 40
epoch 26: 0.36002469062805176, 40
epoch 27: 0.36205941438674927, 40
epoch 28: 0.3643134832382202, 40
epoch 29: 0.3659118711948395, 40
e